# Исследование надёжности заёмщиков

В рамках данного проекта мною были проанализированы историчесике данные киентов банка, которые воспользовались услугами кредитования.

Целью работы было выявить, как семейное положение заемщика и наличие у него детей влияет на факт погашения кредита в срок.

План выполнения:
1. Первичный анализ исходной таблицы и поиск ошибок в данных.
2. Проработка ошибок путем выявления аномалий, пропусков и дубликатов.
3. Категоризация данных, расчет корреляции и выводы.
4. Общий вывод.

## Этап 1. Получение данных и изучение общей инфорамации.

Изучим данные, предоставленные кредитным отделом банка.
Для начала прочитаем и сохраним файл data_1.csv в переменной **credits** и выведем на экран первые 5 строк таблицы.

In [1]:
import pandas as pd
credits = pd.read_csv('data_1.csv')

credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Итак, в таблице представлено 12 колонок.
- children - количество детей;
- days employed - количество отработанных дней;
- dob years - полных лет;
- education - образование;
- education id - идентификационный номер образования;
- family status - семейное положение;
- family status id - идентификационный номер семейного положения;
- gender - пол;
- income type - вид дохода;
- debt - долг;
- total income - суммарный доход;
- purpose - цель кредита.

Уже на первый взгляд можно сказать, что в таблице есть недочеты. Например, колонка **days_employed** содержит отрицательные значения, а в столбце **education** значения прописаны разным регистром, что может в дальнейшем затруднить группировку. 

Выведем на экран общую информации о таблице методом **info()**.

In [2]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Исходя из информации выше, можем сделать вывод, что:
- в исходной таблице содержутся данные о 21525 клиентах банка,
- в колонке **days_employed** указан некорректный тип данных,
- общее количество значений в столбцах отличается, то есть в некоторых строках массива есть пропуски.

Для дальнейшего анализа и рассчета корреляции необходимо устранить имеющиеся ошибки и привести данные в порядок.

## Этап 2. Предобработка данных

### 1. Определение и заполнение пропусков.

На первом этапе мы выяснили, что в таблице есть столбцы с пропущенными значениями. 
Выведем на экран количество пргопусков в тех колонках, где они есть.

Для этого введем переменную **null_count**, значение которой равняется количеству пропущенных значений в каждом столбце, а затем передадим значение этой переменной в цикл для вывода на экран трех параметров: наименование столбца, количество пропусков, процент пропусков в общем массиве.

In [3]:
null_count = credits.isnull().any().values

for n in credits.columns[null_count]:
    print('количество пропусков в {} - {}, доля - {:.2%}'.format(n, credits[n].isnull().sum(), credits[n].isnull().sum()/len(credits)))

количество пропусков в days_employed - 2174, доля - 10.10%
количество пропусков в total_income - 2174, доля - 10.10%


Так как доля пропущенных значений в обоих столбцах составляет более 10% от общего количества строк в таблице, их удаление может повлиять на дальнейший результат. Решать эту проблему будем путем замены пропусков.

В столбце с количеством отработанных дней, заменим пропуски опираясь на возраста клиента.
Возьмем за начало рабочего стажа возраст 18 лет, а за конец - возраст на момент формарования таблицы. Получаем, что стаж равен **(количество полных лет - 18 лет) x 365 дней в году**.
Заменем строки с пропусками на результат данного выражения.

In [4]:
credits['days_employed'] = credits['days_employed'].fillna((credits['dob_years'] - 18)*365)

Теперь необходимо разобраться с суммарным доходом.

Заменим пропуски в графе **total_income** на "заглушки" в виде единиц, что поможет нам сохранить и идентифицировать эти данные в дальнейшем.

Перед тем как осуществить замену, проверим, нет ли сейчас в этой колонке единиц.

In [5]:
credits['total_income'].min()

20667.26379327158

Минимальная сумма суммарного дохода - 20667 рублей. Наши единицы мы с легкостью сможем "распознать" при необходимости. Для замены воспользуемся методом **fillna()** и сразу проверим, остались ли в таблице пропуски.

In [6]:
credits['total_income'] = credits['total_income'].fillna(1)

credits.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Мы избавили нашу таблицу от пропусков! Можно двигаться дальше.

### 2. Замена типа данных

Помимо пропусков, на первом этапе работы так же выявили, что в таблице не все типы данных указаны корректно, а именно - колонка **days_employed**, формат данных в которой должен быть целочисленным.
Переведем данные в столбце из типа **float()** в **int()**.

In [7]:
credits['days_employed'] = credits['days_employed'].astype(int)

credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Теперь все типы данных в порядке. Проверим, есть ли в данных аномалии, которые не бросились в глаза на первом этапе, вызвав метод **describe()**.

In [8]:
credits.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,57624.076609,43.293380,0.817236,0.972544,0.080883,1.505129e+05
std,1.381587,134510.114800,12.574584,0.548138,1.420324,0.272661,1.098971e+05
min,-1.000000,-18388.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,0.000000,-2521.000000,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,-984.000000,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,3650.000000,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Опираясь на полученную таблицу, выявили следующие ошибки (будем описывать их и сразу же исправлять).
1. Минимальное значение в графе **children** равно -1. Скорее всего это ошибка ввода данных. Заменим "-1" на "1" методом **replace()**.

In [9]:
credits['children'] = credits['children'].replace(-1, 1)

2. Максимальное значение в графе children равно 20, что, конечно, тоже маловероятно. Предположим, что это так же ошибка ввода данных. Заменим "20" на "2" по аналогии с предыдущим пунктом.

In [10]:
credits['children'] = credits['children'].replace(20, 2)

С этим столбцом разобрались! Идем дальше.

3. В столбце **days_employed** так же есть отрицательные значения. Посчитаем сколько их.

In [11]:
credits.loc[credits.loc[:,'days_employed'] < 0]['days_employed'].count()

15916

Таких значений больше половины. Очевидно, что это ошибка в обработке данных, поэтому воспользуемся методом **abs()**, чтобы взять все числа в столбце по модулю.

In [12]:
credits['days_employed'] = credits['days_employed'].abs()

4. Максимальное значение в столбце days_employed тоже вызывает вопросы. 401755 дней - это примерно 1100 рабочих лет. Жутковато. Приведем эти цифры к более реальным.

Рассчитаем максимально возможное значение для этого столбца. Возьмем самый большой возраст из нашей таблицы - 75 лет, вычтем из него 18 лет (примерное начало работы человека) и умножим на 365 дней в году: 

                           (75 лет - 18 лет) * 365 дней/год = 20805 рабочих дней

Это максимальная отметка, которую могут принимать значения из колонки days_employed. Заменим все некорректные данные, которые привышают 20805 формулой, которую уже использовали при удалении пропусков.

In [13]:
worked_days = (credits.loc[:,'dob_years'] - 18) * 365
credits.loc[credits.days_employed > 20805, 'days_employed'] = worked_days

Проверим, к чему привели все наши манипуляции с таблицей.

In [14]:
credits.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,5091.808084,43.293380,0.817236,0.972544,0.080883,1.505129e+05
std,0.755528,5506.929423,12.574584,0.548138,1.420324,0.272661,1.098971e+05
min,0.000000,-6570.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,0.000000,1022.000000,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,2555.000000,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,7665.000000,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,5.000000,20440.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбце **days_employed** снова появилось как минимум одно отрицательное значение.

Очевидно, это случилось после того как мы заменили некорректно высокие значения в графе со стажем работы формулой **(credits.loc[:,'dob_years'] - 18) * 365**.
Связано это с тем, что в графе dob_years есть нулевые значения.

Проверим сколько в таблце таких строк.

In [15]:
# Посчитаем кол-во отрицательных значений в столбце days_employed
print(credits.loc[credits.loc[:,'days_employed'] < 0]['days_employed'].count())

# Посчитаем кол-во нулевых значений в столбце dob_years
print(credits.loc[credits.loc[:,'dob_years'] == 0]['dob_years'].count())

17
101


В разрезе всего массива выбросов не так много. Заменим выбросы в столбце **days_employed** на единицы, а в столбце **dob_years** на моду.

In [16]:
credits.loc[credits.days_employed < 0, 'days_employed'] = 1
print(credits.loc[credits.loc[:,'days_employed'] < 0]['days_employed'].count())

credits.loc[credits.dob_years == 0, 'dob_years'] = credits['dob_years'].mode()
print(credits.loc[credits.loc[:,'dob_years'] == 0]['dob_years'].count())

0
0


Отлчино!

Последнее, что бросается в глаза - часть данных в строках **education** и **family_status** записаны в разных регистрах. Это может осложнить аналитику в дальнейшем, например, при группировке, поэтому приведем их к нижнему регистру при помощи функции **lower()**.

In [17]:
credits['education'] = credits['education'].str.lower()
credits['family_status'] = credits['family_status'].str.lower()

На данном этапе мы привели таблицу к оптимальному для дальнейшего исследования виду. 

В будущем необходимо ввести стандартную систему внесенния информации, чтобы не смотря на возможное появление технических ошибок, избежать хотя бы ошибки вызванные некорректным введением данных. 

Рекомендуется предоставлять пользователям шаблоны заполнения, в том числе информировать о необходимости вносить текстовую информацию только в **нижнем регистре**, а при введении сумм исключать **тире**, которые на выходе фигурируют как отрицательные значения.

### 4. Обработка дубликатов

Следующий шаг - поиск и удаление дубликатов. Сперва посчитаем сколько в нашей таблице дублирующих строк. Для этого испольуем комбинацию методов **duplicated()** и **sum()**.

In [18]:
credits.duplicated().sum()

71

Удалим все выявленные дубликаты методом **drop_duplicates()**.

In [19]:
# Удаляем дубликаты 
credits = credits.drop_duplicates().reset_index(drop = True)

# Проверяем себя
credits.duplicated().sum()

0

Среди причин возникновения такого рода недочетов в данных можно выделить:
1. человеческий фактор, такой как ошибки пользователей при внесение информации или ошибки сотрудников банка,которые могут возникнуть, например, в случае повторного обращение заемщика.
2. технический сбой при выгрузке данных в том случае, например, если таблица формировалась посредством соединения данных из разных источников.

### 5. Лемматизация

Кредиты, предоставленные в таблице, являются целевыми. Рассмотрим сколько уникальных целей кредита содержится в таблице и какое количество клиентов по каждой из них. Для этого сгруппируем строки по целям методом **groupby()**.

In [20]:
purpose_grouped = credits.groupby('purpose').count()
purpose_grouped

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income
purpose,,,,,,,,,,,
автомобили,478,478,476,478,478,478,478,478,478,478,478
автомобиль,494,494,490,494,494,494,494,494,494,494,494
высшее образование,452,452,449,452,452,452,452,452,452,452,452
дополнительное образование,460,460,458,460,460,460,460,460,460,460,460
жилье,646,646,640,646,646,646,646,646,646,646,646
заняться высшим образованием,496,496,493,496,496,496,496,496,496,496,496
заняться образованием,408,408,407,408,408,408,408,408,408,408,408
на покупку автомобиля,471,471,466,471,471,471,471,471,471,471,471
на покупку подержанного автомобиля,478,478,478,478,478,478,478,478,478,478,478


Цели одинаковые по смыслу, но разные по написанию дублируются. Так, например, видим, что по меньшей мере 5 из них относятся к покупле автомобиля.

Для решения это проблемы воспользуемся **лемматизацией** и импортируем библиотеку **pymystem3**.

In [21]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [22]:
from pymystem3 import Mystem
m = Mystem()

Напишем функцию, на вход в которую будет попадать каждая строчка колонки **purpose**. Далее применем к ним метод **lemmatize()** и сформируем условия.

In [23]:
def keywords(i):
    lemmas = m.lemmatize((i['purpose']))
    if 'автомобиль' in lemmas:
                         return 'автомобиль'
    if 'образование' in lemmas:
                         return 'образование'
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
                         return 'недвижимость'
    if 'свадьба' in lemmas:
                         return 'свадьба'

Полученный результат, используя метод **apply()**, сохраним в новой колонке **keyword**, чтобы не изменять исходные данные. Результат изменений проверим, выведя на экран первые 5 строк.

In [24]:
credits['keyword'] = credits.apply(keywords, axis=1)

credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,keyword
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,12775,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба


Чтобы понять, все ли цели кредита мы учли при построении функции, проверим сколько раз в новой колонке **keyword** встречается каждая из них и выведем результат. 

In [25]:
credits['keyword'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: keyword, dtype: int64

Отлично, пропусков нет. Теперь, после добавления столбца **keywards**, мы без труда можем оценить, на что именно люди брали денежные средства в банке. Так, например, самой частой целью кредитования являются операции с недвижимостью, составляющие почти 50% всех кредитов.

## Этап 3. Категоризация данных, расчет корреляции и выводы.

Теперь, когда мы привели все данные таблицы в порядок можно приступить к анализу данных для ответа на главный вопрос: **влияет ли семейное положение и количество детей клиента на погашение кредита в срок**.

In [26]:
# Проверяем, какое количество заемщиков имеют долг по кредиту
credits['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

### 1. Зависимость между семейным положением и прсрочками по кредиту.

Теперь рассмотрим как эти показатели распределяются между клиентами в разрезе их семейного положения.

Выведем по каждому семейному положению общее количество клиентов, просрочивших платежи и долю "должников".

In [27]:
# Считаем общее количество клиентов в разрезе семейного положения
family_status_grouped_count = credits.groupby('family_status')['debt'].count()
print(family_status_grouped_count)
print()

# Считаем количество клиентов, просрочивших кредитные выплаты
family_status_grouped_sum = credits.groupby('family_status')['debt'].sum()
print(family_status_grouped_sum)
print()

# Считаем долю клиентов, просрочивших кредитные выплаты
family_status_share = family_status_grouped_sum / family_status_grouped_count
print(family_status_share.map('{:,.3f}'.format))

family_status
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
не женат / не замужем     2810
Name: debt, dtype: int64

family_status
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
не женат / не замужем    274
Name: debt, dtype: int64

family_status
в разводе                0.071
вдовец / вдова           0.066
гражданский брак         0.093
женат / замужем          0.075
не женат / не замужем    0.098
Name: debt, dtype: object


Категория людей, которой на первый взгляд принадлежит самое большое количество просрочек, в процентном соотношении всего лишь на третьем месте.
Люди, не состоящие в браке, чаще других просрочивают выплаты по кредиту.
Предполагаю, что отсутсвие общего семейного бюджета или "незрелость" в процессе его ведения может стать причиной задолженностей, в то время как люди в браке имеют "подушку безопасности" в виде общего дохода, а люди после брака - способны более рационально распределять ресурсы и меньше задерживать выплаты.

### 2. Зависимость между наличием детей и просрочками по кредиту.

Проанализируем по аналогии с семейным положеним графу **children**.

In [28]:
children_grouped_count = credits.groupby('children')['debt'].count()
print(children_grouped_count)
print()

children_grouped_sum = credits.groupby('children')['debt'].sum()
print(children_grouped_sum)
print()

children_share = children_grouped_sum / children_grouped_count
print(children_share.map('{:,.3f}'.format))

children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: debt, dtype: int64

children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

children
0    0.075
1    0.092
2    0.095
3    0.082
4    0.098
5    0.000
Name: debt, dtype: object


Ситуация схожа с пердыдущей - большое количество просрочек в рамках одной категории вовсе не означает, что большинство из нее не платит в срок. 

Проверим как наличие детей в целом или их отсутствие влияет на задоженности по кредиту. Для этого вызовем функцию **check_for_children**, с помощью которой создадим новый столбец **having_children**, в котором будет два значения: **есть дети** и **нет детей**. Также в качестве третьего значения выделим клиентов с пятью детьми и не будем их учитывать, так как они очень сильно "выбиваются" из общего массива данных.

In [29]:
def children_group(i):
    child = i['children']
    if child == 0:
        return 'нет детей'
    if child == 5:
        return 'аномалия'
    else:
        return 'есть дети'

In [30]:
credits['having_children'] = credits.apply(children_group, axis = 1)
credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,keyword,having_children
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,есть дети
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,есть дети
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,есть дети
4,0,12775,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей


Теперь еще раз сгруппируем данные по просрочкам, но уже по новому столбцу **having_children**.

In [31]:
having_children_count = credits.groupby('having_children')['debt'].count()
print(having_children_count)
print()

having_children_sum = credits.groupby('having_children')['debt'].sum()
print(having_children_sum)
print()

having_children_mean = credits.groupby('having_children')['debt'].mean()
print(having_children_mean.map('{:,.3f}'.format))

having_children
аномалия         9
есть дети     7354
нет детей    14091
Name: debt, dtype: int64

having_children
аномалия        0
есть дети     678
нет детей    1063
Name: debt, dtype: int64

having_children
аномалия     0.000
есть дети    0.092
нет детей    0.075
Name: debt, dtype: object


Клиенты с детьми чаще являются задолжниками, нежели те,у кого их нет. 
Вероятно, причина в том, что наличие детей в семьях провоцирует определенные финансовые расходы и порой непредвиденные. Это может осложнять процесс выплат по кредиту.

### 3. Зависимость между семейным положением + наличием детей и просрочками по кредиту.

Теперь попробуем рассмотреть совокупность влияния двух рассмотренных выше параметров: семейнго пложения и наличия детей. Предварительно объединим данные следующим образом:
 - 1 группа - не женат / не замужем или гражданский брак;
 - 2 группа - женат / замужем ;
 - 3 группа - в разводе или вдовец / вдова.

Такие группы выделили на основе предыдущих расчетов:
 - во-первых, по схожести средних величин, рассчитанных выше: 0.093 и 0.097 для первой группы, 0.075 - для второй, 0.071 и 0.066 - для третьей;
 - во-вторых, с логической точки зрения: 1 группа - люди до брака, 2 группа - люди в браке, 3 группа - "после" брака.

Снова создадим дополнительную колонку **family** для присвоения одной из трех категорий.

In [32]:
# Прописываем функцию для категоризации семейного положения
def family_status_grouped(i):
    status = i['family_status']
    if (status == 'не женат / не замужем') or (status == 'гражданский брак'):
                    return 'до брака'
    if status == 'женат / замужем':
                    return 'в браке'
    else:
                return 'после брака'

In [33]:
# Добавляем в таблицу столбец с семейным положением
credits['type_of_family'] = credits.apply(family_status_grouped, axis = 1)
credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,keyword,having_children,type_of_family
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,есть дети,в браке
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,есть дети,в браке
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей,в браке
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,есть дети,в браке
4,0,12775,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей,до брака


In [34]:
type_of_family_mean = credits.groupby('type_of_family')['debt'].mean()
type_of_family_mean.map('{:,.3f}'.format)

type_of_family
в браке        0.075
до брака       0.095
после брака    0.069
Name: debt, dtype: object

Так гораздо удобней. Явно видно, что люди вне брака чаше являются должниками, нежели остальные категории клиентов.

Теперь в двух последних столбцах мы имеем необходимые параметры для оценки корреляции. Объединим их.

In [35]:
# Прописываем функцию для категоризации по семейному положению и наличию детей
def search_for_a_reason(i):
    child = i['having_children']
    status = i['type_of_family']

    if child == 'есть дети':
        if status == 'до брака':
            return 'до брака, есть дети'
        if status == 'в браке':
            return 'в браке, есть дети'
        if status == 'после брака':
            return 'после брака, есть дети'
    if child == 'нет детей':
        if status == 'до брака':
            return 'до брака, нет детей'
        if status == 'в браке':
            return 'в браке, нет детей'
        if status == 'после брака':
            return 'после брака, нет детей'
    return 'аномалия'

In [36]:
credits['search_for_a_reason'] = credits.apply(search_for_a_reason, axis = 1)
credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,keyword,having_children,type_of_family,search_for_a_reason
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,есть дети,в браке,"в браке, есть дети"
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,есть дети,в браке,"в браке, есть дети"
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей,в браке,"в браке, нет детей"
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,есть дети,в браке,"в браке, есть дети"
4,0,12775,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей,до брака,"до брака, нет детей"


Рассчитаем корреляцию по каждой категории.

In [37]:
# Общее количество клиентов
correlation_count = credits.groupby('search_for_a_reason')['debt'].count()
print(correlation_count)
print()

# Клиенты просрочившие выплаты
correlation_sum = credits.groupby('search_for_a_reason')['debt'].sum()
print(correlation_sum)
print()

# Доля клиентов-должников
correlation_mean = credits.groupby('search_for_a_reason')['debt'].mean()
print(correlation_mean.map('{:,.3f}'.format))

search_for_a_reason
аномалия                     9
в браке, есть дети        4864
в браке, нет детей        7468
до брака, есть дети       1967
до брака, нет детей       4992
после брака, есть дети     523
после брака, нет детей    1631
Name: debt, dtype: int64

search_for_a_reason
аномалия                    0
в браке, есть дети        415
в браке, нет детей        516
до брака, есть дети       223
до брака, нет детей       439
после брака, есть дети     40
после брака, нет детей    108
Name: debt, dtype: int64

search_for_a_reason
аномалия                  0.000
в браке, есть дети        0.085
в браке, нет детей        0.069
до брака, есть дети       0.113
до брака, нет детей       0.088
после брака, есть дети    0.076
после брака, нет детей    0.066
Name: debt, dtype: object


Для удобства отсортируем средние значения по убыванию с помощью **sort_values()**.

In [38]:
correlation_mean.map('{:,.3f}'.format).sort_values(ascending = False)

search_for_a_reason
до брака, есть дети       0.113
до брака, нет детей       0.088
в браке, есть дети        0.085
после брака, есть дети    0.076
в браке, нет детей        0.069
после брака, нет детей    0.066
аномалия                  0.000
Name: debt, dtype: object

Наибольшее число просрочек по кредиту наблюдается у категории "до брака, есть дети" и составляет более 10%. Самой ответственной можно смело назвать категорию "после брака, нет детей". У них просрочек почти два раза меньше, чем у первых.

### 4. Зависимость между уровнем дохода и просрочками по кредиту.

Дополнительно проверим влияние еще нескольких параметров на просрочки по кредиту.
Категоризируем данные и найдем минимальную, среднюю и максимальную суммы дохода.

In [39]:
income_min = credits['total_income'].min()
print(income_min)

income_mean = credits['total_income'].mean()
print(income_mean)

income_max = credits['total_income'].max()
print(income_max)

1.0
151011.05029506548
2265604.028722744


Выделим трехступенчатую категоризацию по сумме дохода:
 - меньше 100 000 рублей
 - от 100 000 до 200 000 рублей
 - от 200 000 рублей
Нужно помнить, что значене "1" мы присвоили пустым строкам самостоятельно, поэтому не будем их учитывать.

In [40]:
def income_grouped(i):
    income = i['total_income']
    if income == 1:
                    return 'аномалия'
    if 1 < income < 100000:
                    return 'до 100 000'
    if income > 200000:
                    return 'больше 200000'
    else:
                return 'от 100 000 до 200 000'

In [41]:
credits['income'] = credits.apply(income_grouped, axis = 1)
credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,keyword,having_children,type_of_family,search_for_a_reason,income
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,есть дети,в браке,"в браке, есть дети",больше 200000
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,есть дети,в браке,"в браке, есть дети",от 100 000 до 200 000
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей,в браке,"в браке, нет детей",от 100 000 до 200 000
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,есть дети,в браке,"в браке, есть дети",больше 200000
4,0,12775,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей,до брака,"до брака, нет детей",от 100 000 до 200 000


In [42]:
income_mean = credits.groupby('income')['debt'].mean()
income_mean.map('{:,.3f}'.format)

income
аномалия                 0.081
больше 200000            0.071
до 100 000               0.079
от 100 000 до 200 000    0.087
Name: debt, dtype: object

Значения распределились довольно равномерно. Можно сделать предположение, что уровень дохода незначительно влияет на факт задолженности по кредиту. 
Однако разница все же есть: клиенты с доходом свыше 200 000 рублей реже остальных задерживают выплаты. Высокая платежеспособность позволяет своевременно возвращать денежные средства банку.
Чаще других "должниками" становятся люди со средним уровнем дохода (в нашем случае это 100 000 - 200 000 рублей). Возможно, их общий заработок позволяет кредитоваться на крупные денежные суммы, но своевременно вносить ежемесячные выплаты и гасить такие крупные задолженности пока не представляется возможным.

### 5. Зависимость между целью кредита и просрочками по кредиту.

Предворительно с помощью лемматизации мы уже категоризировали данные и сохранили их в столбец **keyword**, поэтому достаточно сгруппировать данные по этому столбцу и посчитать среднюю.

In [43]:
keyword_mean = credits.groupby('keyword')['debt'].mean()
keyword_mean.map('{:,.3f}'.format)

keyword
автомобиль      0.094
недвижимость    0.072
образование     0.092
свадьба         0.080
Name: debt, dtype: object

Наименьшее число просроченных выплат у людей, которые берут кредит на покупку недвижимости, вероятно, чем масштабнее суммы, тем серьезней подход к делу. 
Просрочки по выплатам кредита за автомобиль самые частые. Возможно, тут тоже имеет место масштаб кредита.

## Этап 4. Общий вывод

В ходе работы для дальнейшего анализа были оптимизированы данные предоставленной таблицы: 
- заменены пустые ячейки; 
- исправлены типы даннных в столбцах;
- удалены дубликаты;
- скорректированы аномальные значения.

Посредством дальнейшего анализа было выявлено:
1. наличие детей у клиента увеличивает вероятность просрочки кредита;
2. клиенты, не состоящие в браке, чаще задерживают выплаты;
3. уровень дохода не оказывает существенного влияния на выплаты по кредитам, однако ежемесячный доход свыше 200000 рублей с большей вероятностью гарантирует выплату кредита в срок;
4. клиенты, взявшие кредит на недвижимость, погашают задолженность с большей регулярностью, а те, кто брал его на покупку/ремонт автомобиля - лидеры по просрочкам.

Кроме того, совмщенный анализ стлбцов **дети** и **семейное положение** позволил выявить самую "безответственную" категорию клиентов - это люди до брака (не женаты/замужем или проживающие в гражданском браке) и с детьми. А самые пунктуальные - люди в разводе или вдовцы/вдовы без детей.

до брака, есть дети       0.113371
до брака, нет детей       0.087941
в браке, есть дети        0.085321
после брака, есть дети    0.076482
в браке, нет детей        0.069095
после брака, нет детей    0.066217

#### Рекомендации. 
При рассмотрении потенциальных клиентов на кредитование и одобрении суммы займа необходимо учитыватьсь несколько важных факторов.
1. Семейное положение клиента. Вероятность возвращения кредита в срок выше у людей, состоящих на данный момент или состоявщих ранее в браке. Людям, выплачивающим кредит "в одиночку" сложнее погашать займы вовремя.
2. Наличие детей у клиентов. Заемщики без детей - самые ответственные. Исходя из анализа данных, можно рекомендовать этот сегмент населения как самый платежеспособный, особенно если они состоят или состояли ранее в браке.
3. Уровень дохода. Высокая заработная плата (свыше 200 000 рублей) - показатель, характеризующий самых ответственных заемщиков.
4. Цель кредита. Масштабные займы, такие как операции с недвижимостью, имеют самую высокую степень возвратности в срок.

Таким образом, "идеальный заемкик" - это клиент, состоящий (состоявший) в браке, без детей, с доходом свыше 200 000 рубле и, нацеленный на покупку недвижимости.